In [1]:
!sudo apt-get update -y
!sudo apt-get install python3.8 python3.8-distutils python3.8-dev -y

!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.11 2
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
!sudo update-alternatives --set python3 /usr/bin/python3.8
!wget https://bootstrap.pypa.io/pip/3.8/get-pip.py -O get-pip.py
!python3.8 get-pip.py

Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,065 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,32

In [2]:
!python --version

Python 3.8.20


In [3]:
# clone SadTalker
!git clone https://github.com/OpenTalker/SadTalker.git
%cd SadTalker

# install dependencies
!python3.8 -m pip install --upgrade pip setuptools wheel
!python3.8 -m pip install -r requirements.txt

# download pretrained checkpoints
!bash scripts/download_models.sh

Cloning into 'SadTalker'...
remote: Enumerating objects: 1605, done.
remote: Counting objects: 100% (545/545), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 1605 (delta 470), reused 445 (delta 445), pack-reused 1060 (from 1)
Receiving objects: 100% (1605/1605), 92.20 MiB | 17.03 MiB/s, done.
Resolving deltas: 100% (882/882), done.
/content/SadTalker
  Preparing metadata (setup.py) ... done
  Using cached torch-2.4.1-cp38-cp38-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached networkx-3.1-py3-none-any.whl.metadata (5.3 kB)
  Preparing metadata (setup.py) ... done
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
INFO: pip is looking at multiple versions of scikit-learn to determine which version is compatible with other requirements. This could take a while.
  Using cached sympy-1.13.3-py3-none-any.whl.metadata (12 k

In [4]:
!python3.8 -m pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 155.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 120.4 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.4.1
    Uninstalling torch-2.4.1:
      Successfully uninstalled torch-2.4.1
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.19.1
    Uninstalling torchvision-0.19.1:
      Successfully uninstalled torchvision-0.19.1


In [8]:
import sys
!{sys.executable} -m pip install flask pyngrok --ignore-installed blinker

  Using cached werkzeug-3.0.6-py3-none-any.whl.metadata (3.7 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached PyYAML-6.0.3-cp38-cp38-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (2.1 kB)
  Using cached zipp-3.20.2-py3-none-any.whl.metadata (3.7 kB)
  Using cached MarkupSafe-2.1.5-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.0 kB)
Using cached click-8.1.8-py3-none-any.whl (98 kB)
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
Using cached PyYAML-6.0.3-cp38-cp38-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl (806 kB)
Using cached werkzeug-3.0.6-py3-none-any.whl (227 kB)
Using cached MarkupSafe-2.1.5-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26 kB)
Using cached zipp-3.20.2-py3-none-any.whl (9.2 kB)


In [9]:
import sys, importlib, os
print("sys.executable:", sys.executable)
print("python version:", sys.version)
!{sys.executable} -m pip show pyngrok
!{sys.executable} -m pip list | grep pyngrok || true
print("find_spec:", importlib.util.find_spec("pyngrok"))

sys.executable: /usr/bin/python3
python version: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
Name: pyngrok
Version: 7.2.12
Summary: A Python wrapper for ngrok
Home-page: 
Author: 
Author-email: 
License: MIT License
        
        Copyright (c) 2018-2025 Alex Laird
        
        Permission is hereby granted, free of charge, to any person obtaining a copy
        of this software and associated documentation files (the "Software"), to deal
        in the Software without restriction, including without limitation the rights
        to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
        copies of the Software, and to permit persons to whom the Software is
        furnished to do so, subject to the following conditions:
        
        The above copyright notice and this permission notice shall be included in all
        copies or substantial portions of the Software.
        
        THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EX

In [10]:
import sys
sys.path.insert(0, "/usr/local/lib/python3.8/dist-packages")
from pyngrok import ngrok
print("ok", ngrok)

ok <module 'pyngrok.ngrok' from '/usr/local/lib/python3.8/dist-packages/pyngrok/ngrok.py'>


In [11]:
from flask import Flask, request, jsonify, send_file
from pyngrok import ngrok
import subprocess
import uuid
import shutil
import base64
import io

app = Flask(__name__)

@app.route('/lipsync', methods=['POST'])
def create_lipsync():
    try:
        # Decode base64 files
        data = request.json
        image_data = base64.b64decode(data['image'])
        audio_data = base64.b64decode(data['audio'])

        # Save files
        session_id = str(uuid.uuid4())
        temp_dir = f"/content/temp_{session_id}"
        os.makedirs(temp_dir, exist_ok=True)

        image_path = f"{temp_dir}/source.jpg"
        audio_path = f"{temp_dir}/audio.wav"
        results_dir = f"{temp_dir}/results"

        with open(image_path, 'wb') as f:
            f.write(image_data)
        with open(audio_path, 'wb') as f:
            f.write(audio_data)

        # Run SadTalker
        cmd = [
            'python', 'inference.py',
            '--driven_audio', audio_path,
            '--source_image', image_path,
            '--result_dir', results_dir,
            '--still',
            '--preprocess', 'crop',
            '--enhancer', 'gfpgan'
        ]

        subprocess.run(cmd, cwd='/content/SadTalker', check=True)

        # Find output video
        for root, dirs, files in os.walk(results_dir):
            for file in files:
                if file.endswith('.mp4'):
                    output_path = os.path.join(root, file)
                    with open(output_path, 'rb') as f:
                        video_data = base64.b64encode(f.read()).decode()

                    # Cleanup
                    shutil.rmtree(temp_dir)

                    return jsonify({
                        'success': True,
                        'video': video_data
                    })

        return jsonify({'success': False, 'error': 'No output generated'}), 500

    except Exception as e:
        return jsonify({'success': False, 'error': str(e)}), 500

@app.route('/health', methods=['GET'])
def health():
    return jsonify({'status': 'healthy', 'service': 'sadtalker-colab'})

In [12]:
from pyngrok import ngrok
import json

# Set your ngrok auth token (get from ngrok.com)
ngrok.set_auth_token("2zy1iAMFuUNxSNf8ZF1F0dYJsJO_3FjsdqffVjGAbRz7Ga1oe")  # Optional but recommended

# Start tunnel
public_url = ngrok.connect(8003)
print(f"Public URL: {public_url}")
print(f"API Endpoint: {public_url}/lipsync")

# Save URL to file for easy access
with open('/content/api_url.txt', 'w') as f:
    f.write(str(public_url))

Public URL: NgrokTunnel: "https://2fbb0295385c.ngrok-free.app" -> "http://localhost:8003"
API Endpoint: NgrokTunnel: "https://2fbb0295385c.ngrok-free.app" -> "http://localhost:8003"/lipsync


In [13]:
app.run(port=8003)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:8003
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [04/Oct/2025 01:31:06] "POST /lipsync HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Oct/2025 01:32:11] "POST /lipsync HTTP/1.1" 500 -
